In [2]:
#import library
from PIL import Image
import io
import urllib
import uuid
from concurrent.futures import ThreadPoolExecutor
from functools import partial

import numpy as np
from tqdm import tqdm
from datasets import load_dataset
from datasets.utils.file_utils import get_datasets_user_agent
import matplotlib.pyplot as plt
import tensorflow as tf

#import other packages for dataset store
import pandas as pd
from datasets import Dataset, DatasetDict, load_from_disk

In [3]:
test_dataset = load_dataset('csv', data_files='test.csv')

Using custom data configuration default-d6533c8cfe3effc0


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to C:/Users/Zhiwen Yan/.cache/huggingface/datasets/csv/default-d6533c8cfe3effc0/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
test_dataset

DatasetDict({
    train: Dataset({
        features: ['Id', 'ImageURL', 'ISO639-3'],
        num_rows: 200
    })
})

In [ ]:
#split test data based on hau, tha, kir
total_test = pd.read_csv("test.csv")
test_hau = total_test[total_test['ISO639-3']=='hau']
test_tha = total_test[total_test['ISO639-3']=='tha']
test_kir = total_test[total_test['ISO639-3']=='kir']

In [5]:
#actual download image from url
USER_AGENT = get_datasets_user_agent()

def fetch_single_image(image_url, timeout=None, retries=0):
    request = urllib.request.Request(
        image_url,
        data=None,
        headers={"user-agent": USER_AGENT},
    )
    with urllib.request.urlopen(request, timeout=timeout) as req:
        if 'png' in image_url:
          png = Image.open(io.BytesIO(req.read())).convert('RGBA')
          png.load() # required for png.split()
          background = Image.new("RGB", png.size, (255, 255, 255))
          background.paste(png, mask=png.split()[3]) # 3 is the alpha channel
          image_id = str(uuid.uuid4()) # confused about image_id here?
          image_path = "images_test/" + image_id + ".jpg"
          background.save(image_path, 'JPEG', quality=80)
        else:
          image = Image.open(io.BytesIO(req.read()))
          image_id = str(uuid.uuid4())
          image_path = "images_test/" + image_id + ".jpg"
          image.save(image_path)
    return image_path

def fetch_images(batch, num_threads, timeout=None, retries=3):
    fetch_single_image_with_args = partial(fetch_single_image, timeout=timeout, retries=retries)
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        batch["image_path"] = list(executor.map(fetch_single_image_with_args, batch["ImageURL"]))
    return batch

num_threads = 20
test_dataset = test_dataset.map(fetch_images, batched=True, batch_size=100, fn_kwargs={"num_threads": num_threads})

Parameter 'function'=<function fetch_images at 0x0000026CE28FC168> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/2 [00:00<?, ?ba/s]

In [8]:
#save dataset to local
test_dataset.save_to_disk("dataset_test")

In [9]:
#reload dataset
test_dataset = load_from_disk("dataset_test")